In [1]:
!pip install pydot graphviz opencv-python

In [2]:
import os
import glob
import errno
import shutil
import cv2
import random

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K

from tensorflow.keras import regularizers
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Concatenate, BatchNormalization, Average, Convolution2D, MaxPooling2D, Activation, GlobalMaxPooling2D, AveragePooling2D, GlobalAveragePooling2D 
from tensorflow.keras.models import Model, Sequential
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras import initializers
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2

from sklearn.metrics import classification_report

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.14.0


In [3]:
if tf.__version__.startswith("1."):
    val_acc_name = "val_acc"
    acc_name = "acc"
else:
    val_acc_name = "val_accuracy"
    acc_name = "accuracy"

In [4]:
K.image_data_format()

'channels_last'

In [5]:
class_names = ["cat", "dog"]
GLOBAL_RANDOM_STATE = 159034912
NEW_WIDTH = 299
batch_size = 128 # 32 # 64 # 128
image_shape = (299, 299, 3)

In [6]:
def create_or_check_path(path):
    try:
        os.makedirs(path)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

In [7]:
catimgs_filepaths_list = glob.glob(os.path.join("data","PetImages","Cat_resized_%d" % NEW_WIDTH, "*.*"))
random.seed(GLOBAL_RANDOM_STATE)
random.shuffle(catimgs_filepaths_list)
catimgs_filepaths_list[:5]

[]

In [8]:
dogimgs_filepaths_list = glob.glob(os.path.join("data","PetImages","Dog_resized_%d" % NEW_WIDTH, "*.*"))
random.seed(GLOBAL_RANDOM_STATE)
random.shuffle(dogimgs_filepaths_list)
dogimgs_filepaths_list[:5]

[]

In [9]:
output_data_dir = os.path.join("data","PetImages", "flow_from_directory", "width_%d" % NEW_WIDTH)
create_or_check_path(output_data_dir)
output_data_dir

'data/PetImages/flow_from_directory/width_299'

In [10]:
train_dir = os.path.join(output_data_dir, 'train')
create_or_check_path(train_dir)

validation_dir = os.path.join(output_data_dir, 'validation')
create_or_check_path(validation_dir)

test_dir = os.path.join(output_data_dir, 'test')
create_or_check_path(test_dir)

train_cats_dir = os.path.join(train_dir, 'cats')
create_or_check_path(train_cats_dir)

train_dogs_dir = os.path.join(train_dir, 'dogs')
create_or_check_path(train_dogs_dir)

validation_cats_dir = os.path.join(validation_dir, 'cats')
create_or_check_path(validation_cats_dir)

validation_dogs_dir = os.path.join(validation_dir, 'dogs')
create_or_check_path(validation_dogs_dir)

test_cats_dir = os.path.join(test_dir, 'cats')
create_or_check_path(test_cats_dir)

test_dogs_dir = os.path.join(test_dir, 'dogs')
create_or_check_path(test_dogs_dir)

In [11]:
cats_train_count = int(len(catimgs_filepaths_list) * 0.7)
cats_valid_count = int(len(catimgs_filepaths_list) * 0.15)
cats_test_count = len(catimgs_filepaths_list) - cats_train_count - cats_valid_count

dogs_train_count = int(len(dogimgs_filepaths_list) * 0.7)
dogs_valid_count = int(len(dogimgs_filepaths_list) * 0.15)
dogs_test_count = len(dogimgs_filepaths_list) - dogs_train_count - dogs_valid_count

In [12]:
# fnames = ['cat.{}.jpg'.format(i) for i in range (cats_train_count)]
# for fname in fnames:
#   src = catimgs_filepaths_list.pop()
#   dst = os.path.join(train_cats_dir, fname)
#   shutil.copyfile(src, dst)

# fnames = ['cat.{}.jpg'.format(i) for i in range (cats_valid_count)]
# for fname in fnames:
#   src = catimgs_filepaths_list.pop()
#   dst = os.path.join(validation_cats_dir, fname)
#   shutil.copyfile(src, dst)

# fnames = ['cat.{}.jpg'.format(i) for i in range (cats_test_count)]
# for i, fname in enumerate(fnames):
#   src = catimgs_filepaths_list[i]
#   dst = os.path.join(test_cats_dir, fname)
#   shutil.copyfile(src, dst)

In [13]:
# fnames = ['dog.{}.jpg'.format(i) for i in range (dogs_train_count)]
# for fname in fnames:
#   src = dogimgs_filepaths_list.pop()
#   dst = os.path.join(train_dogs_dir, fname)
#   shutil.copyfile(src, dst)

# fnames = ['dog.{}.jpg'.format(i) for i in range (dogs_valid_count)]
# for fname in fnames:
#   src = dogimgs_filepaths_list.pop()
#   dst = os.path.join(validation_dogs_dir, fname)
#   shutil.copyfile(src, dst)

# fnames = ['dog.{}.jpg'.format(i) for i in range (dogs_test_count)]
# for i, fname in enumerate(fnames):
#   src = dogimgs_filepaths_list[i]
#   dst = os.path.join(test_dogs_dir, fname)
#   shutil.copyfile(src, dst)

In [14]:
len(os.listdir(train_cats_dir))

8733

In [15]:
len(os.listdir(validation_cats_dir))

1871

In [16]:
len(os.listdir(test_cats_dir))

1872

In [17]:
len(os.listdir(train_dogs_dir))

8729

In [18]:
len(os.listdir(validation_dogs_dir))

1870

In [19]:
len(os.listdir(test_dogs_dir))

1871

In [20]:
train_data_count = len(glob.glob(os.path.join(train_dir, "**", "*.*")))
train_data_count

17462

In [21]:
validation_data_count = len(glob.glob(os.path.join(validation_dir, "**", "*.*")))
validation_data_count

3741

In [22]:
def make_model():
    inputs = Input(shape=image_shape, name="Input")
    base_model = InceptionResNetV2(include_top=False, 
                               input_shape=image_shape, 
                               input_tensor=inputs, 
                               weights='imagenet', 
                               classes=2)
    base_model.trainable = False
    batch_norm1 = BatchNormalization()(base_model.output)
    flat1 = Flatten()(batch_norm1)
    dense1 = Dense(100, activation="relu")(flat1)
    dense2 = Dense(len(class_names), activation="softmax")(dense1)

    model = Model(inputs=[inputs], outputs=[dense2])
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [23]:
# Масштабируем изображения, применяем аугментацию.
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    zoom_range=0.15,
    rescale=1./255
    )

# Тестовые изображения только масштабируем.
test_datagen = ImageDataGenerator(rescale=1./255)

In [24]:
# Обучающие данные
# Приводим к одному размеру, создаем бинарные метки.
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(NEW_WIDTH, NEW_WIDTH), 
                                                    batch_size=batch_size, 
                                                    class_mode='categorical')

Found 17462 images belonging to 2 classes.


In [25]:
# Валидационные данные
# Приводим к одному размеру, создаем бинарные метки.
validation_generator = test_datagen.flow_from_directory(validation_dir, 
                                                    target_size=(NEW_WIDTH, NEW_WIDTH), 
                                                    batch_size=batch_size, 
                                                    class_mode='categorical')

Found 3741 images belonging to 2 classes.


In [26]:
# Тестовые данные
# Приводим к одному размеру, создаем бинарные метки.
test_generator = test_datagen.flow_from_directory(test_dir, 
                                                    target_size=(NEW_WIDTH, NEW_WIDTH), 
                                                    batch_size=batch_size, 
                                                    class_mode='categorical')

Found 3743 images belonging to 2 classes.


In [27]:
%%time
K.clear_session()

model = make_model()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
CPU times: user 12.9 s, sys: 517 ms, total: 13.4 s
Wall time: 13.6 s


In [28]:
len(model.layers)

784

In [29]:
checkpoint = ModelCheckpoint('cat_dogs_with_InceptionResNetV2_flow_from_directory.hdf5',
                            monitor=val_acc_name,
                            save_best_only=True,
                            period=1,
                            verbose=1)
early_stopping_callback = EarlyStopping(monitor=val_acc_name, 
                                        min_delta=1e-6, 
                                        patience=50, 
                                        verbose=1, 
                                        mode='auto', 
                                        baseline=None, 
                                        restore_best_weights=True)
learning_rate_reduction = ReduceLROnPlateau(monitor=val_acc_name,
                                           patience=5,
                                           verbose=1,
                                           factor=0.5,
                                           min_lr=0.00000001)

In [30]:
# # history = model.fit(X_train, y_train,
# #                    epochs = 999999999999, batch_size=batch_size,
# #                    validation_data=(X_valid, y_valid),
# #                    verbose=1,
# #                    callbacks=[checkpoint, learning_rate_reduction, early_stopping_callback])

# # Обучаем модель при помощи генератора пакетов
# history = model.fit_generator(train_generator, 
#                               steps_per_epoch=train_data_count // batch_size, 
#                               epochs=30, 
#                               validation_data=validation_generator, 
#                               validation_steps=validation_data_count,
#                              callbacks=[checkpoint, learning_rate_reduction, early_stopping_callback])

In [31]:
# history = model.fit(X_train, y_train,
#                    epochs = 999999999999, batch_size=batch_size,
#                    validation_data=(X_valid, y_valid),
#                    verbose=1,
#                    callbacks=[checkpoint, learning_rate_reduction, early_stopping_callback])

# Обучаем модель при помощи генератора пакетов
history = model.fit_generator(train_generator, 
                              steps_per_epoch=train_data_count // batch_size, 
                              epochs=30, 
                              validation_data=validation_generator, 
                              validation_steps=validation_data_count // batch_size,
                             callbacks=[checkpoint, learning_rate_reduction, early_stopping_callback],
                             workers=3,
                             max_queue_size=10,
                             use_multiprocessing=False)

Epoch 1/30
135/136 [============================>.] - ETA: 2s - loss: 0.4835 - acc: 0.9520
Epoch 00001: val_acc improved from -inf to 0.92484, saving model to cat_dogs_with_InceptionResNetV2_flow_from_directory.hdf5
136/136 [==============================] - 426s 3s/step - loss: 0.4821 - acc: 0.9521 - val_loss: 0.3344 - val_acc: 0.9248
Epoch 2/30
135/136 [============================>.] - ETA: 1s - loss: 0.0977 - acc: 0.9788
Epoch 00002: val_acc improved from 0.92484 to 0.94450, saving model to cat_dogs_with_InceptionResNetV2_flow_from_directory.hdf5
136/136 [==============================] - 305s 2s/step - loss: 0.0971 - acc: 0.9789 - val_loss: 0.2486 - val_acc: 0.9445
Epoch 3/30
 39/136 [=======>......................] - ETA: 3:15 - loss: 0.1277 - acc: 0.9778

KeyboardInterrupt: 